In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
import sys

In [2]:
sc = SparkContext("local[*]", "NFL")

In [3]:
rdd = sc.textFile('../in/PlayerTrackData.csv')

In [4]:
# show header
rdd.take(1)

['PlayKey,time,event,x,y,dir,dis,o,s']

In [5]:
# remove header
rdd = rdd.filter(lambda x: 'PlayKey' not in x)

In [6]:
# split the col
rdd = rdd.map(lambda x: x.split(','))

In [14]:
# convert to dataframe object
sqlContext = SQLContext(sc)
df = rdd.toDF(['PlayKey','time','event','x','y','dir','dis','o','s'])

# if df want to use rdd function
# just df.rdd.xxx()

In [15]:
df.select('PlayKey','s').show()

+---------+----+
|  PlayKey|   s|
+---------+----+
|26624-1-1|0.13|
|26624-1-1|0.12|
|26624-1-1|0.12|
|26624-1-1| 0.1|
|26624-1-1|0.09|
|26624-1-1|0.07|
|26624-1-1|0.05|
|26624-1-1|0.02|
|26624-1-1|0.01|
|26624-1-1|0.01|
|26624-1-1| 0.0|
|26624-1-1|0.01|
|26624-1-1|0.01|
|26624-1-1|0.01|
|26624-1-1| 0.0|
|26624-1-1|0.01|
|26624-1-1| 0.0|
|26624-1-1| 0.0|
|26624-1-1| 0.0|
|26624-1-1| 0.0|
+---------+----+
only showing top 20 rows



In [30]:
# a.printSchema()
player_mean_speed_df = df.rdd.map(lambda x: [x[0].split('-')[0], float(x[-1])]).toDF(["playerKey","s"])

# remove extreme small speed
player_mean_speed_df = player_mean_speed_df.rdd.filter(lambda x:x.s >= 1).toDF(["playerKey","s"])

# get average speed
player_mean_speed = player_mean_speed_df.groupBy('playerKey').agg(f.mean("s").alias('mean'))

In [ ]:
result = player_mean_speed.collect()

In [ ]:
def writeResult(res):
    with open('../out/player_avg_speed_gte_1.txt','w') as f:
        for line in res:
            f.write('{} {}\n'.format(line[0],line[1]))

In [ ]:
writeResult(result)